<a href="https://colab.research.google.com/github/Berlingrad/MLinCyberSecurityProject/blob/BasicModelRun/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DNN Back Door Detector

## Preprocessing

###Data Download

In [1]:
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

!gdown https://drive.google.com/uc?id=1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
!gdown https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
!gdown https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV

!mv /content/*.h5 /content/CSAW-HackML-2020/data

##!wget https://raw.githubusercontent.com/bolunwang/backdoor/master/visualizer.py

fatal: destination path 'CSAW-HackML-2020' already exists and is not an empty directory.
Downloading...
From: https://drive.google.com/uc?id=1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
To: /content/sunglasses_poisoned_data.h5
398MB [00:02, 195MB/s]
Downloading...
From: https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
To: /content/clean_validation_data.h5
716MB [00:02, 247MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV
To: /content/clean_test_data.h5
398MB [00:08, 45.3MB/s]


In [2]:
import os
import gdown
import numpy as np
import random
import h5py
import glob2

import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

if not tf.config.experimental.list_physical_devices('GPU'):
    raise Exception("Please change runtime type to GPU")


In [3]:
def load_dataset(data_filename, keys=None):
    dataset = {}
    with h5py.File(data_filename, 'r') as hf:
        if keys is None:
            for name in hf:
                dataset[name] = np.array(hf.get(name))
        else:
            for name in keys:
                dataset[name] = np.array(hf.get(name))

    return dataset

def load_models(model_dir):
    models = []
    for net in glob2.glob("/content/CSAW-HackML-2020/models/*net.h5"):
        weights = glob2.glob(net.split("_")[0]+"*weights.h5")[0]
        print(net.split('/')[-1])
        m = load_model(net)
        m.load_weights(weights)

        models.append(m)
    return models

###Loading Data and Reshaping

In [4]:
# Loading Data
SG_poisoned_path = "/content/CSAW-HackML-2020/data/sunglasses_poisoned_data.h5"
SG_poisoned = load_dataset(SG_poisoned_path)
print(SG_poisoned.keys())
print(SG_poisoned['data'].shape)
print(SG_poisoned['label'].shape)

dict_keys(['data', 'label'])
(12830, 3, 55, 47)
(12830,)


In [5]:
clean_validation_path = "/content/CSAW-HackML-2020/data/clean_validation_data.h5"
valid_clean = load_dataset(clean_validation_path)
valid_clean.keys()
print(valid_clean['data'].shape)
print(valid_clean['label'].shape)

(11547, 3, 55, 47)
(11547,)


In [6]:
# Reshaping the data
SG_poisoned['data'] = SG_poisoned['data'].transpose((0,2,3,1))
valid_clean['data'] = valid_clean['data'].transpose((0,2,3,1))

###Data Parameters

In [7]:
##TODO

##SG_nClass = 
##vc_nClass = 

###Load Models

In [8]:
model_dir = "/content/CSAW-HackML-2020/models"

models = load_models(model_dir)

anonymous_bd_net.h5
sunglasses_bd_net.h5
multi_trigger_multi_target_bd_net.h5


#### Predicting:

In [9]:
# Using sunglasses_bd_net.h5 to predict the sunglasses_poisoned_data.h5
Y_pred_sunglasses_using_sunglasses_bn = np.argmax(models[1].predict(SG_poisoned['data']), axis = 1)
accuracy = np.mean(np.equal(Y_pred_sunglasses_using_sunglasses_bn, SG_poisoned['label']))*100
accuracy

99.94544037412315

In [10]:
Y_pred_val_sunglasses_using_sunglasses_bn = np.argmax(models[1].predict(valid_clean['data']), axis = 1)
accuracy = np.mean(np.equal(Y_pred_val_sunglasses_using_sunglasses_bn, valid_clean['label']))*100
accuracy

42.25339915129471

In [22]:
# Same thing using existing eval.py
!python3 CSAW-HackML-2020/eval.py CSAW-HackML-2020/data/sunglasses_poisoned_data.h5 CSAW-HackML-2020/models/sunglasses_bd_net.h5

2020-12-19 09:21:04.696978: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-19 09:21:17.951458: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-19 09:21:17.953599: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-19 09:21:17.958369: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-19 09:21:17.958724: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-12-19 09:21:17.958756: I tensorflow/stream_executor/platform/default/dso_loade

In [11]:
models[1].summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
____________________________________________________________________________________________

In [12]:
models[1].get_layer('activation_1')

In [13]:
from keras.models import Model

layer_name = 'activation_1'
act_layer_model = Model(inputs=models[1].input,
                                 outputs=models[1].get_layer(layer_name).output)
act_layer_model_out = np.array(act_layer_model.predict(valid_clean['data']))

In [14]:
act_layer_model_out.shape

(11547, 160)

In [15]:
act_layer_model_out

array([[18328.363 ,  3369.2559,     0.    , ...,  1788.5098,     0.    ,
         7089.8496],
       [    0.    ,     0.    ,     0.    , ...,  1044.509 ,     0.    ,
        15550.116 ],
       [    0.    ,     0.    ,     0.    , ...,     0.    ,     0.    ,
            0.    ],
       ...,
       [    0.    ,     0.    ,     0.    , ...,     0.    ,     0.    ,
            0.    ],
       [ 2243.541 ,  2730.6855,     0.    , ...,     0.    ,     0.    ,
         7055.434 ],
       [    0.    ,  2279.0913,   559.8897, ...,     0.    ,  2567.4998,
         9197.23  ]], dtype=float32)

In [16]:
act_layer_model_out.sort()
act_layer_model_out

array([[    0.   ,     0.   ,     0.   , ..., 39825.258, 44517.965,
        48114.28 ],
       [    0.   ,     0.   ,     0.   , ..., 11913.555, 15550.116,
        15642.143],
       [    0.   ,     0.   ,     0.   , ..., 23995.299, 29360.316,
        35755.21 ],
       ...,
       [    0.   ,     0.   ,     0.   , ..., 22552.074, 24341.379,
        26666.203],
       [    0.   ,     0.   ,     0.   , ..., 21104.203, 21746.684,
        23233.727],
       [    0.   ,     0.   ,     0.   , ..., 22805.14 , 22992.977,
        33424.023]], dtype=float32)

In [17]:
!pip install -q tensorflow-model-optimization

## Pruning the Sunglasses BadNet and predicting Validation data

In [18]:
# pruning 
import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(models[1])

model_for_pruning.summary()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv_1 (Pru (None, 52, 44, 20)   1942        input[0][0]                      
__________________________________________________________________________________________________
prune_low_magnitude_pool_1 (Pru (None, 26, 22, 20)   1           prune_low_magnitude_conv_1[0][0] 
__________________________________________________________________________________________________
prune_low_magnitude_conv_2 (Pru (None, 24, 20, 40)   14442       prune_low_magnitude_pool_1[0][0] 
____________________________________________________________________________________________

In [19]:
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [20]:
import tempfile
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(valid_clean['data'], valid_clean['label'],
                  batch_size=valid_clean['data'].shape[0], epochs=25, validation_split=0.2,
                  callbacks=callbacks)

Epoch 1/25
1/1 [==============================] - 6s 6s/step - loss: 1438.7886 - accuracy: 0.3166 - val_loss: 742.1019 - val_accuracy: 0.4996
Epoch 2/25
1/1 [==============================] - 1s 544ms/step - loss: 726.4903 - accuracy: 0.5106 - val_loss: 453.6267 - val_accuracy: 0.6372
Epoch 3/25
1/1 [==============================] - 1s 510ms/step - loss: 428.0554 - accuracy: 0.6473 - val_loss: 342.3535 - val_accuracy: 0.6939
Epoch 4/25
1/1 [==============================] - 1s 537ms/step - loss: 301.3499 - accuracy: 0.7130 - val_loss: 300.1232 - val_accuracy: 0.7208
Epoch 5/25
1/1 [==============================] - 1s 547ms/step - loss: 246.5214 - accuracy: 0.7404 - val_loss: 284.9702 - val_accuracy: 0.7364
Epoch 6/25
1/1 [==============================] - 1s 545ms/step - loss: 217.9815 - accuracy: 0.7589 - val_loss: 276.2768 - val_accuracy: 0.7429
Epoch 7/25
1/1 [==============================] - 0s 485ms/step - loss: 196.5296 - accuracy: 0.7793 - val_loss: 264.8918 - val_accuracy: 0

In [26]:
model_for_pruning.save("CSAW-HackML-2020/models/pruned_sunglasses_bd_net.h5")

In [29]:
pruned_model_predictions = np.argmax(model_for_pruning.predict(valid_clean['data']), axis = 1)
accuracy = np.mean(np.equal(pruned_model_predictions, valid_clean['label']))*100
accuracy

94.9857105741751